In [22]:
import tensorflow as tf
tf.test.gpu_device_name()

'/device:GPU:0'

In [23]:
# Load the Drive helper and mount
from google.colab import drive

# This will prompt for authorization.
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [24]:
import pandas as pd
Data =  pd.read_json('/content/drive/My Drive/Sarcasm Detection/Arabic News Headlines Dataset.json', orient = 'split')
Data.head()

,headlines,topic,length,is_sarcastic
0,طالب يصبح متعلّماً بعد دراسته وحدة عن القائد ف...,Society,12,1
1,سموحة يقلص الفارق ويسجل هدفا ثانيا في شباك الم...,Sports,11,0
2,ليتها الحدود – الولايات المتحدة تهدد بفرض عقوب...,Misc,17,1
3,وزيرة الهجرة تتواصل مع سفير مصر في نيوزيلندا ل...,None,14,0
4,ليتها الحدود – مصر تقرر إرسال أعضاء السائح الب...,Misc,12,1


In [0]:
X = Data['headlines']
Labels = Data['is_sarcastic']

In [26]:
import numpy as np
max_sequence_length = np.max(Data['length'])
print("max length is:", max_sequence_length)

max length is: 34


In [27]:
import keras
from keras.preprocessing.text import Tokenizer

tokenizer = Tokenizer(num_words=30000)
tokenizer.fit_on_texts(X)
sequences = tokenizer.texts_to_sequences(X)

word_index = tokenizer.word_index   # a dictionary of each word and its index
print('Found %s unique tokens.' % len(word_index))

Found 20595 unique tokens.


In [28]:
from keras.preprocessing.sequence import pad_sequences
data = pad_sequences(sequences, maxlen=max_sequence_length)
print('Shape of data tensor:', data.shape)

Shape of data tensor: (6669, 34)


In [29]:
from keras.utils import to_categorical
labels = to_categorical(np.asarray(Labels))  ## one hot of the output
print('Shape of label tensor:', labels.shape)

Shape of label tensor: (6669, 2)


In [0]:
import pickle as p
fastText = p.load(open('/content/drive/My Drive/Sarcasm Detection/FastText_Wiki_Embeddings.p', 'rb'))

In [31]:
# prepare embedding matrix  (matrix of embeddings of the words in our dataset)
num_words = len(word_index) + 1
# initialize the np array
unKnown = list()
embedding_matrix = np.zeros((num_words, 300))
l=0
# find the embeddings from the pre trained embedding matrix of fast text wiki
for word, i in word_index.items():
    embedding_vector = fastText.get(word)
    if embedding_vector is not None:
        l+=1
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector
    else:
      unKnown.append(word)
print("the number of words found in fast text embeddings =", l)
print("the number of words not found in fast text embeddings =", 20595- l)

the number of words found in fast text embeddings = 17289
the number of words not found in fast text embeddings = 3306


In [0]:
from keras.initializers import Constant
from keras.layers import Input, Embedding, Conv2D, GlobalMaxPooling2D, Dense, Dropout, LSTM
from keras.models import Model

In [0]:
# shuffle the data

indices = np.arange(data.shape[0])
np.random.shuffle(indices)
data = data[indices]
labels = labels[indices]


## split the data into training, testing

nb_validation_samples = int(0.25* data.shape[0])

x_train = data[:-nb_validation_samples]
y_train = labels[:-nb_validation_samples]
x_test = data[-nb_validation_samples:]
y_test = labels[-nb_validation_samples:]

In [0]:
embedding_layer = Embedding(num_words,
                            300,
                            embeddings_initializer=Constant(embedding_matrix),
                            input_length=max_sequence_length,
                            trainable=False)

In [0]:
# the model

sequence_input = Input(shape=(max_sequence_length,), dtype='int32')             ## input layer
embedded_sequences = embedding_layer(sequence_input)                            ## embeddings layer
X = LSTM(128, return_sequences=True)(embedded_sequences)
X = Dropout(0.5)(X)
X = LSTM(128, return_sequences=True)(X)
X = Dropout(0.5)(X)
X = keras.layers.core.Reshape((34,128,1))(X)
x1 = Conv2D(2, (2, 100), activation='relu')(X)                 ## conv 1
x1 = GlobalMaxPooling2D()(x1)                                                   ## max 1
x2 = Conv2D(2, (3, 100), activation='relu')(X)                 ## conv 2
x2 = GlobalMaxPooling2D()(x2)                                                   ## max 2
x3 = Conv2D(2, (4, 100), activation='relu')(X)                 ## conv 3
x3 = GlobalMaxPooling2D()(x3)                                                   ## max 3
x  = keras.layers.concatenate([x1, x2, x3])                                     ## concatenate
preds = Dense(2, activation='softmax')(x)                                       ## softmax layer

model = Model(sequence_input, preds)

model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['acc'])

In [0]:
from keras.utils import plot_model
plot_model(model, to_file='/content/drive/My Drive/Sarcasm Detection/Plots/LSTM_CNN.png')

In [37]:
model.fit(x_train, y_train, batch_size=32, epochs=15)

Epoch 1/15
5002/5002 [==============================] - 18s 4ms/step - loss: 0.5269 - acc: 0.7641
Epoch 2/15
5002/5002 [==============================] - 16s 3ms/step - loss: 0.4114 - acc: 0.8756
Epoch 3/15
5002/5002 [==============================] - 16s 3ms/step - loss: 0.3442 - acc: 0.8990
Epoch 4/15
5002/5002 [==============================] - 15s 3ms/step - loss: 0.2959 - acc: 0.9146
Epoch 5/15
5002/5002 [==============================] - 17s 3ms/step - loss: 0.2466 - acc: 0.9332
Epoch 6/15
5002/5002 [==============================] - 16s 3ms/step - loss: 0.2156 - acc: 0.9386
Epoch 7/15
5002/5002 [==============================] - 15s 3ms/step - loss: 0.1842 - acc: 0.9460
Epoch 8/15
5002/5002 [==============================] - 15s 3ms/step - loss: 0.1599 - acc: 0.9546
Epoch 9/15
5002/5002 [==============================] - 15s 3ms/step - loss: 0.1435 - acc: 0.9594
Epoch 10/15
5002/5002 [==============================] - 16s 3ms/step - loss: 0.1275 - acc: 0.9652
Epoch 11/15
5002/50

In [38]:
loss, accuracy = model.evaluate(x_test, y_test)
print('test loss is:', loss)
print('test accuracy is: ', accuracy)

1667/1667 [==============================] - 3s 2ms/step
test loss is: 0.4838410771011091
test accuracy is:  0.9124175164967007


In [0]:
from keras import backend as K
def f1(y_true, y_pred):
    def recall(y_true, y_pred):
        
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

    def precision(y_true, y_pred):
        
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision
    precision = precision(y_true, y_pred)
    recall = recall(y_true, y_pred)
    f1 = 2*((precision*recall)/(precision+recall+K.epsilon()))
    
    return f1
    
model.compile(loss='categorical_crossentropy',
          optimizer= "rmsprop",
          metrics=[f1])

In [40]:
model.fit(x_train, y_train, batch_size=32, epochs=15)

Epoch 1/15
5002/5002 [==============================] - 19s 4ms/step - loss: 0.0638 - f1: 0.9816
Epoch 2/15
5002/5002 [==============================] - 16s 3ms/step - loss: 0.0648 - f1: 0.9820
Epoch 3/15
5002/5002 [==============================] - 16s 3ms/step - loss: 0.0547 - f1: 0.9842
Epoch 4/15
5002/5002 [==============================] - 16s 3ms/step - loss: 0.0531 - f1: 0.9838
Epoch 5/15
5002/5002 [==============================] - 18s 4ms/step - loss: 0.0563 - f1: 0.9848
Epoch 6/15
5002/5002 [==============================] - 17s 3ms/step - loss: 0.0562 - f1: 0.9862
Epoch 7/15
5002/5002 [==============================] - 16s 3ms/step - loss: 0.0485 - f1: 0.9876
Epoch 8/15
5002/5002 [==============================] - 16s 3ms/step - loss: 0.0461 - f1: 0.9878
Epoch 9/15
5002/5002 [==============================] - 17s 3ms/step - loss: 0.0371 - f1: 0.9898
Epoch 10/15
5002/5002 [==============================] - 18s 4ms/step - loss: 0.0357 - f1: 0.9912
Epoch 11/15
5002/5002 [======

In [41]:
loss, accuracy = model.evaluate(x_test, y_test)
print('test loss is:', loss)
print('test f1 score is: ', accuracy)

1667/1667 [==============================] - 3s 2ms/step
test loss is: 0.46782735120628005
test f1 score is:  0.9172165005880698


In [42]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 34)           0                                            
__________________________________________________________________________________________________
embedding_2 (Embedding)         (None, 34, 300)      6178800     input_2[0][0]                    
__________________________________________________________________________________________________
lstm_3 (LSTM)                   (None, 34, 128)      219648      embedding_2[0][0]                
__________________________________________________________________________________________________
dropout_3 (Dropout)             (None, 34, 128)      0           lstm_3[0][0]                     
__________________________________________________________________________________________________
lstm_4 (LS